In [1]:
import pandas as pd
import numpy as np

In [33]:
# df_rat = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')

# df_rat = df_rat[
#     (df_rat['NAT.CODIGO'].str.contains('Y0700[1345]')) |
#     (df_rat['NAT.CODIGO'] == 'Y04012')  
# ]

# df_rat.set_index('RAT.NUM_ATIVIDADE', inplace=True)

# df_ratv = pd.read_sql_table('tbl_rat_veiculo', 'sqlite:///gdo.db')
# df_ratv = df_ratv['NUM_ATIVIDADE'].value_counts()
# df_ratv.rename('VIATURAS', inplace=True)
# df_rat = df_rat.join(df_ratv, how='left')
# df_rat['VIATURAS'].fillna(0, inplace=True)
# df_rat['VIATURAS'] = df_rat['VIATURAS'].astype(int)

# df_rate = pd.read_sql_table('tbl_rat_efetivo', 'sqlite:///gdo.db')
# df_rate = df_rate['NUM_ATIVIDADE'].value_counts()
# df_rate.rename('EFETIVO', inplace=True)
# df_rat = df_rat.join(df_rate, how='left')
# df_rat['EFETIVO'].fillna(0, inplace=True)
# df_rat['EFETIVO'] = df_rat['EFETIVO'].astype(int)


df_rat.groupby(['VIATURAS','EFETIVO','NAT.CODIGO']).count().loc[0]



NAT.DESCRICAO  DTA_HRA_INCLUSAO  DTA_INICIO  HRA_INICIO  \
EFETIVO NAT.CODIGO                                                            
1       Y04012                 10                10          10          10   
        Y07001                 22                22          22          22   
2       Y07001                139               139         139         139   
        Y07003                  1                 1           1           1   
        Y07004                  2                 2           2           2   
3       Y04012                  1                 1           1           1   
        Y07001                 25                25          25          25   
        Y07003                  3                 3           3           3   
4       Y04012                  2                 2           2           2   
        Y07001                  5                 5           5           5   
6       Y07001                  1                 1           1           1   
        Y07004                  1                 1           1           1   
7       Y04012                  1                 1           1           1   
10      Y07004                  1                 1           1           1   

                    DTA_TERMINO  HRA_TERMINO  DES_ALVO_EVENTO  DES_LUGAR  \
EFETIVO NAT.CODIGO                                                         
1       Y04012               10           10               10         10   
        Y07001               22           22               22         22   
2       Y07001              139          139              139        139   
        Y07003                1            1                1          1   
        Y07004                2            2                2          2   
3       Y04012                1            1                1          1   
        Y07001               25           25               25         25   
        Y07003                3            3                3          3   
4       Y04012                2            2                2          2   
        Y07001                5            5                5          5   
6       Y07001                1            1                1          1   
        Y07004                1            1                1          1   
7       Y04012                1            1                1          1   
10      Y07004                1            1                1          1   

                    NOM_OPERACAO  COD_UNIDADE_SERVICO  ...  DTA_HRA_INICIO  \
EFETIVO NAT.CODIGO                                     ...                   
1       Y04012                10                   10  ...              10   
        Y07001                22                   22  ...              22   
2       Y07001               139                  139  ...             139   
        Y07003                 1                    1  ...               1   
        Y07004                 2                    2  ...               2   
3       Y04012                 1                    1  ...               1   
        Y07001                25                   25  ...              25   
        Y07003                 3                    3  ...               3   
4       Y04012                 2                    2  ...               2   
        Y07001                 5                    5  ...               5   
6       Y07001                 1                    1  ...               1   
        Y07004                 1                    1  ...               1   
7       Y04012                 1                    1  ...               1   
10      Y07004                 1                    1  ...               1   

                    DTA_HRA_INICIO_DT  DTA_HRA_TERMINO  DTA_HRA_TERMINO_DT  \
EFETIVO NAT.CODIGO                                                           
1       Y04012                     10               10                  10   
        Y07001                     22               22 

In [4]:
df_rat.groupby('SETOR').count()['RAT.NUM_ATIVIDADE']

SETOR
ALTO GOIAS                              894
BOM PASTOR                             4582
CARMO DO CAJURU/SAO GONCALO DO PARA    6376
CENTRO/PADRE EUSTAQUIO                 4815
CLAUDIO                                3546
HIPER CENTRO                           3512
LOURDES/ITATIAIUCU                     2601
NITEROI                                4327
PLANALTO                               3105
PORTO VELHO                            3910
SANTANENSE/MORADA NOVA                 4159
SAO JOSE                               3939
other                                   265
Name: RAT.NUM_ATIVIDADE, dtype: int64

In [5]:
df_rat[    
    (df_rat['NAT.CODIGO'].str.contains('Y0700[1345]')) |
    (df_rat['NAT.CODIGO'] == 'Y04012')    
]['SETOR'].value_counts()

CARMO DO CAJURU/SAO GONCALO DO PARA    2903
NITEROI                                2434
PORTO VELHO                            2369
SAO JOSE                               2324
BOM PASTOR                             1978
HIPER CENTRO                           1823
CLAUDIO                                1522
CENTRO/PADRE EUSTAQUIO                 1476
SANTANENSE/MORADA NOVA                 1247
LOURDES/ITATIAIUCU                     1030
PLANALTO                                924
ALTO GOIAS                              576
Name: SETOR, dtype: int64

In [ ]:
# df_ratv = pd.read_sql_table('tbl_rat_veiculo', 'sqlite:///gdo.db')
# df_ratv = df_ratv['NUM_ATIVIDADE'].value_counts()
# df_ratv.rename('VEICULOS', inplace=True)
# df_rat = df_rat.join(df_ratv, how='left')
# df_rat['VEICULOS'].fillna(0, inplace=True)
# df_rat['VEICULOS'] = df_rat['VEICULOS'].astype(int)

# df_rat = df_rat[
#     (df_rat_e_v['NAT.CODIGO'].str.contains('Y0700[1345]')) |
#     (df_rat_e_v['NAT.CODIGO'] == 'Y04012')  
# ]
# df_rat.groupby(['CIA','SETOR']).count()['index']

In [4]:
# df = pd.DataFrame({
#     'A':[1, 2, 3, 4],
#     'B':[5, 6, 7, 8],
#     'C':[4, 3, 2, 1]
# }, columns=list('ABC'))
# df.to_sql('tbl_teste', 'sqlite:///gdo.db', if_exists='append', index=False)
# df2.to_sql('tbl_teste', 'sqlite:///gdo.db', if_exists='append')
df2 = pd.read_sql('tbl_teste', 'sqlite:///gdo.db')
# df2.index = [0,1,5,3,6]
# df2.set_index('index', inplace=True)
df2

,A,B,C
0,1,5,4
1,2,6,3
2,3,7,2
3,4,8,1
